### Домашнее задание 7 - Студент: Андреа Грилланди

Оценивание - если вы сделали всё вышеперечисленное - 10 баллов. Каждая ошибка - минус 0.5 балла. 

Выложите код к себе на гитхаб и вставьте ссылку в поле ниже (в тетрадке должны быть показатели метрик и ваши комментарии).

#### Нужные библиотеки и функции

In [0]:
# !pip install pymorphy2

In [0]:
import nltk

In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

from pymystem3 import Mystem
m = Mystem()

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)

In [0]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

### Задача 1
1) Векторизуйте тексты с помощью Word2vec модели, обученной самостоятельно, и с помощью модели, взятой с rusvectores (любой). Обучите 2 модели по определению перефразирования на получившихся векторах и проверьте, что работает лучше. 
Word2Vec нужно обучить на отдельном корпусе (не на парафразах). Можно взять данные из семинара или любые другие. 
ВАЖНО: Оценивать модели нужно с помощью кросс-валидации! Метрика - f1.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# mapping = {}

# for line in open(os.path.join(os.getcwd(), 'drive', 'My Drive', 'ru-rnc.map.txt')):
#     ms, ud = line.strip('\n').split()
#     mapping[ms] = ud

In [0]:
# def normalize_mystem(text):
#     tokens = []
#     norm_words = m.analyze(text)
#     for norm_word in norm_words:
#         if 'analysis' not in norm_word:
#             continue
            
#         if not len(norm_word['analysis']):
#             lemma = norm_word['text']
#             pos = 'UNKN'
#         else:
#             lemma = norm_word["analysis"][0]["lex"].lower().strip()
#             pos = norm_word["analysis"][0]["gr"].split(',')[0]
#             pos = pos.split('=')[0].strip()
#         pos = mapping[pos]
#         tokens.append(lemma+'_'+pos)

#     return ' '.join(tokens)

In [0]:
# os.listdir(os.path.join(os.getcwd(), 'drive', 'My Drive'))

In [0]:
# import zipfile
# with zipfile.ZipFile(os.path.join(os.getcwd(), 'drive', 'My Drive', 'paraphraser.zip'), 'r') as zip_ref:
#     zip_ref.extractall(os.path.join(os.getcwd(), 'drive', 'My Drive'))

In [0]:
# corpus_xml = html.fromstring(open(os.path.join(os.getcwd(), 'drive', 'My Drive', 'paraphrases.xml'), 'rb').read())
# texts_1 = []
# texts_2 = []
# classes = []

# for p in corpus_xml.xpath('//paraphrase'):
#     texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
#     texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
#     classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
# data = pd.DataFrame({'label':classes, 'text_1':texts_1, 'text_2':texts_2})

In [0]:
data = pd.read_csv(os.path.join(os.getcwd(), 'drive', 'My Drive', 'data_paraphraser_norm.csv'), encoding='utf-8')

In [0]:
data['text_1_n'] = data['text_1'].apply(normalize)
data['text_2_n'] = data['text_2'].apply(normalize)

In [24]:
data.shape

(7227, 7)

In [25]:
data.head()

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_n,text_2_n
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться_VERB из_ADP сирия_NOUN россиянин_NOU...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,в_ADP москва_NOUN из_ADP сирия_NOUN вернуться_...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия


In [0]:
y = data['label'].values

### Создаем наш корпус

In [0]:
corpus = open(os.path.join(os.getcwd(), 'drive', 'My Drive', 'corpus_hum.txt')).read().splitlines()

corpus_norm = [normalize(text) for text in corpus] 

In [0]:
corpus_norm = [text for text in corpus_norm if text]

### Обучим w2v на наш корпус

In [0]:
w2v = gensim.models.Word2Vec([text.split() for text in corpus_norm], size=50, sg=1)

In [18]:
w2v.most_similar('язык')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('древнееврейский', 0.8238615989685059),
 ('латынь', 0.8062160611152649),
 ('фарси', 0.7985751628875732),
 ('иврит', 0.7945111989974976),
 ('церковно-славянский', 0.775028645992279),
 ('идиш', 0.7736895680427551),
 ('по-арабски', 0.7705888152122498),
 ('латинский', 0.7492571473121643),
 ('двуязычный', 0.7450699806213379),
 ('транслитерация', 0.7412364482879639)]

### Берем предобученную модель tayga-func_upos_skipgram_300_5_2019

In [0]:
# with zipfile.ZipFile(os.path.join(os.getcwd(), 'drive', 'My Drive', 'taiga_model.zip'), 'r') as zip_ref:
#     zip_ref.extractall(os.path.join(os.getcwd(), 'drive', 'My Drive'))

In [0]:
# os.listdir(os.path.join(os.getcwd(), 'drive', 'My Drive'))

In [33]:
# taiga = gensim.models.KeyedVectors.load_word2vec_format(os.path.join(os.getcwd(), 'drive', 'My Drive', 'model.bin'), binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [35]:
taiga.most_similar('февраль_NOUN')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ноябрь_NOUN', 0.9718526601791382),
 ('декабрь_NOUN', 0.9698548316955566),
 ('октябрь_NOUN', 0.9665497541427612),
 ('март_NOUN', 0.9644964933395386),
 ('январь_NOUN', 0.9629086256027222),
 ('апрель_NOUN', 0.9617109298706055),
 ('июль_NOUN', 0.9445033073425293),
 ('август_NOUN', 0.93914395570755),
 ('июнь_NOUN', 0.9278451204299927),
 ('сентябрь_NOUN', 0.9265590906143188)]

### Преобразуем наши данные из перефразования в вектора

In [0]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

#### Преобразование с помощью w2v

In [27]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_n']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_n']), dim))

for i, text in enumerate(data['text_1_n'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_n'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [0]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [29]:
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')

print(cross_val_score(clf, X_text_w2v, y, scoring='f1_micro', cv=5))

[0.45089903 0.49377593 0.48927336 0.40207612 0.36885813]


In [31]:
clf_lr = LogisticRegression(C=1000, class_weight='balanced')
X_scaled = preprocessing.scale(X_text_w2v)
print(cross_val_score(clf_lr, X_scaled, y, scoring='f1_micro', cv=5))

[0.39349931 0.42254495 0.42076125 0.37647059 0.39238754]


#### Преобразование с помощью модель taiga

In [0]:
dim = 50
X_text_1_tg = np.zeros((len(data['text_1_norm']), dim))
X_text_2_tg = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, taiga, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, taiga, dim)

In [0]:
X_text_tg = np.concatenate([X_text_1_tg, X_text_2_tg], axis=1)

In [50]:
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
print(cross_val_score(clf, X_text_tg, y, scoring='f1_micro', cv=5))

[0.35684647 0.40940526 0.23321799 0.35778547 0.35709343]


In [51]:
clf_lr = LogisticRegression(C=1000, class_weight='balanced')
X_scaled = preprocessing.scale(X_text_tg)
print(cross_val_score(clf_lr, X_scaled, y, scoring='f1_micro', cv=5))

[0.35684647 0.35684647 0.35778547 0.35778547 0.35709343]


### Задача 2
2) Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec (свой и  русвекторовский), Fastext. У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары). 

Постройте обучающую выборку из этих близостей . Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру). Попробуйте улучить метрику изменив параметры в методах векторизации.

SVD и NMF применяйте к данным напрямую, а w2w и fastext обучите на отдельном корпусе (как в первой части). 

### Обучение fastText

In [0]:
fast_text = gensim.models.FastText([text.split() for text in corpus_norm], size=50, min_n=4, max_n=8) 

In [53]:
fast_text.most_similar('язык')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('н.м.язык', 0.9943525195121765),
 ('праязык', 0.9461914300918579),
 ('подъязык', 0.9186094999313354),
 ('язык-предок', 0.8882485032081604),
 ('языкова', 0.8626261353492737),
 ('язык-цель', 0.8466439843177795),
 ('метаязык', 0.8370060920715332),
 ('протоязык', 0.8064162135124207),
 ('языковед', 0.7832568287849426),
 ('диалектолог', 0.7538113594055176)]

In [54]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.4, max_features=1000,
                min_df=3, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
X_text_norm = np.concatenate([data['text_1_n'], data['text_2_n']])

In [0]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
X = cv.fit_transform(X_text_norm)

#### Модель SVD

In [66]:
svd = TruncatedSVD(50)
svd.fit(X)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
             random_state=None, tol=0.0)

In [0]:
X_text_1_svd = svd.fit_transform(tfidf.transform(data['text_1_n']))
X_text_2_svd = svd.fit_transform(tfidf.transform(data['text_2_n']))

X_text_svd = np.concatenate([X_text_1_svd, X_text_2_svd], axis=1)

#### Модель NMF

In [68]:
nmf = NMF(50)
nmf.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=50, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
X_text_1_nmf = nmf.transform(tfidf.transform(data['text_1_n']))
X_text_2_nmf = nmf.transform(tfidf.transform(data['text_2_n']))

X_text_nmf = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

#### Модель fastText

In [70]:
dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [0]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

### Вычисление косинусной близости

In [72]:
X_text_1_w2v.shape

(7227, 50)

In [73]:
X_text_w2v.shape

(7227, 100)

In [0]:
# X_text_w2v[1]

In [75]:
cosine_distances(X_text_1_w2v[1].reshape(1, -1), X_text_2_w2v[1].reshape(1, -1))

array([[1.]])

In [0]:
def cos_similarity(matrix_1, matrix_2):
  cos_sim_lst = []
  matrix = np.concatenate([matrix_1, matrix_2], axis=1)
  for i, _ in enumerate(matrix):
    cos_sim = cosine_distances(matrix_1[i].reshape(1, -1), matrix_2[i].reshape(1, -1))
    cos_sim_lst.append(cos_sim)
  return cos_sim_lst

##### Косинусная близость word2vec

In [0]:
data['cos_sim_w2v'] = cos_similarity(X_text_1_w2v, X_text_2_w2v)

##### Косинусная близость taiga

In [0]:
data['cos_sim_taiga'] = cos_similarity(X_text_1_tg, X_text_2_tg)

##### Косинусная близость fastText

In [0]:
data['cos_sim_ft'] = cos_similarity(X_text_1_ft, X_text_2_ft)

##### Косинусная близость SVD

In [0]:
data['cos_sim_svd'] = cos_similarity(X_text_1_svd, X_text_2_svd)

##### Косинусная близость NMF

In [0]:
data['cos_sim_nmf'] = cos_similarity(X_text_1_nmf, X_text_2_nmf)

In [90]:
data[:30]

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_n,text_2_n,text_1_notnorm,text_2_notnorm,cos_sim_w2v,cos_sim_taiga,cos_sim_ft,cos_sim_svd,cos_sim_nmf
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,полицейским разрешат стрелять на поражение по ...,полиции могут разрешить стрелять по хулиганам ...,[[1.0]],[[1.0]],[[0.23127745300379465]],[[1.5167255366574488]],[[1.0]]
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,право полицейских на проникновение в жилище ре...,правила внесудебного проникновения полицейских...,[[1.0]],[[1.0]],[[0.21128477999455342]],[[1.2796345871528942]],[[1.0]]
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,президент египта ввел чрезвычайное положение в...,власти египта угрожают ввести в стране чрезвыч...,[[1.0]],[[1.0]],[[0.4057655013449778]],[[1.9335628681107064]],[[1.0]]
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться_VERB из_ADP сирия_NOUN россиянин_NOU...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия,вернувшихся из сирии россиян волнует вопрос тр...,самолеты мчс вывезут россиян из разрушенной сирии,[[1.0]],[[1.0]],[[0.5623136002783127]],[[1.0583921397083726]],[[1.0]]
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,в_ADP москва_NOUN из_ADP сирия_NOUN вернуться_...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия,в москву из сирии вернулись 2 самолета мчс с р...,самолеты мчс вывезут россиян из разрушенной сирии,[[1.0]],[[1.0]],[[0.5648112352713865]],[[1.6521676258562137]],[[1.0]]
5,1,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,пристав_NOUN собирать_VERB отпечаток_NOUN пале...,пристав_NOUN снимать_VERB отпечаток_NOUN палец...,пристав собрать отпечаток палец российский дол...,пристав снять отпечаток палец злостный неплате...,приставы соберут отпечатки пальцев российских ...,приставы снимут отпечатки пальцев у злостных н...,[[1.0]],[[1.0]],[[0.2761561967552667]],[[1.2409248206942942]],[[1.0]]
6,-1,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,на_ADP саратовский_ADJ дебошир_NOUN с_ADP борт...,саратовский_ADJ дебошир_NOUN отказываться_VERB...,саратовский дебошир борт самолёт москва хургад...,саратовский дебошир отказываться возвращаться ...,на саратовского дебошира с борта самолета моск...,саратовский дебошир отказывается возвращаться ...,[[1.0]],[[1.0]],[[0.5540815127930185]],[[0.5132558063611543]],[[1.0]]
7,0,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,цик_NOUN хотеть_VERB отказываться_VERB от_ADP ...,цик_NOUN мочь_VERB отказываться_VERB от_ADP эл...,цик хотеть отказаться электронный система подс...,цик отказаться электронный средство подсчёт голос,цик хочет отказаться от электронной системы по...,цик может отказаться от электронных средств по...,[[1.0]],[[1.0]],[[0.09871484883111359]],[[1.8652235219563569]],[[1.0]]
8,-1,Суд Петербурга оставил на потом дело о гибели ...,Ло

### Последний шаг

In [0]:
X_cos_sim = data[['cos_sim_w2v', 'cos_sim_taiga', 'cos_sim_ft', 'cos_sim_svd', 'cos_sim_nmf']].values

In [0]:
train_X, valid_X, train_y, valid_y = train_test_split(X_cos_sim, y, random_state=1)

In [94]:
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
print(cross_val_score(clf, X_cos_sim, y, scoring='f1_micro', cv=5))

[0.49239281 0.53526971 0.54256055 0.45536332 0.45605536]


In [95]:
clf_lr = LogisticRegression(C=1000, class_weight='balanced')
X_scaled = preprocessing.scale(X_cos_sim)
print(cross_val_score(clf_lr, X_scaled, y, scoring='f1_micro', cv=5))

[0.45850622 0.49723375 0.51211073 0.4366782  0.41868512]
